In [10]:
import re
from urllib.error import HTTPError
import pandas as pd
from textblob import TextBlob
from textblob.exceptions import NotTranslated

FILE_NAME = 'magazine_tweets.csv'

In [11]:
company_tweets = pd.read_csv(FILE_NAME, sep=',', names=['id', 'text'])
company_tweets.drop_duplicates(subset='text', inplace=True)
company_tweets.sort_values('id', inplace=True)

In [13]:
company_tweets_clean = pd.DataFrame(columns=('id', 'date', 'text', 'text_en', 'sentiment_polarity', 'sentiment_subjectivity'))
try:
    for i in company_tweets.index:
        if company_tweets.at[i, 'id'] > 4158:
            full_text = company_tweets.at[i, 'text']
            date, tweet = full_text.split("Text=")
            date_splitted = date.split(' ')
            
            if date_splitted[1] == "Sep":
                new_date_format = "2019-09-{:s} {:s}".format(date_splitted[2], date_splitted[3])
            elif date_splitted[1] == "Oct":
                new_date_format = "2019-10-{:s} {:s}".format(date_splitted[2], date_splitted[3])
            else:
                new_date_format = "2019-11-{:s} {:s}".format(date_splitted[2], date_splitted[3])
            
            tweet = ''.join(tweet.split('\\n'))
            clean_tweet = ' '.join(re.sub("(@[A-Za-z0-9-ZÀ-ÿ]+)|([^0-9A-Za-z-ZÀ-ÿ \t])|(\w+://\S+)", " ", tweet).split())
            
            phrase = TextBlob(clean_tweet)
            try:
                translate = TextBlob(str(phrase.translate(from_lang='pt', to='en_US')))
                single_row = {
                             'id': company_tweets.at[i, 'id'],
                             'date': new_date_format,
                             'text': str(phrase),
                             'text_en': str(translate),
                             'sentiment_polarity': translate.sentiment.polarity,
                             'sentiment_subjectivity': translate.sentiment.subjectivity
                              }
                company_tweets_clean = company_tweets_clean.append(single_row, ignore_index=True)
            except NotTranslated:
                    print("Invalid tweet {:s}!".format(str(phrase)))
except HTTPError:
    print("Google translate API: daily limit usage has been reached!")

Google translate API: daily limit usage has been reached!


In [14]:
final = pd.read_csv(FILE_NAME.split('.')[0] + "_clean.csv", sep=',')
final.drop(final.columns[[0]], axis=1, inplace=True)

In [7]:
merged_data = pd.concat([final, company_tweets_clean])
print(merged_data.to_string())

       id                 date                                               text                                            text_en  sentiment_polarity  sentiment_subjectivity
0    4159  2019-10-21 01:43:48                                   tá na hora né lu                                 it's time right lu            0.285714                0.535714
1    4160  2019-10-21 01:56:53                    Eu gosto do programa do Faustão                      I like the program of Faustão            0.000000                0.000000
2    4161  2019-10-21 09:24:21  não faço a mínima ideia do que significa Prova...  I have no idea what it means I'll probably eat...            0.000000                0.000000
3    4162  2019-10-21 10:17:20  nossa uma viagem com celular novo seria tudo d...  our a trip with new cell phone would be all go...            0.418182                0.527273
4    4163  2019-10-21 11:24:51  reclamacao magalu indignado Bom dia Lu tudo be...  magalu indignado good morning Lu

In [8]:
merged_data.to_csv(FILE_NAME.split('.')[0] + "_clean.csv", sep=',', encoding='utf-8')
